In [1]:
#Random Policy
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
import timeit
import random 

def random_lunar(env, num_episodes, max_steps_per_episode):

    episode_rewards_history = []
    #last_episode_video_path = './Documents/Reinforcement_Learning/RL_proj/rl-group-assignment'
    
    for episode in range(num_episodes):
        state = env.reset()[0]
        episode_rewards = []

        for step in range(max_steps_per_episode):
            num_actions = env.action_space.n
            action = random.randint(0, num_actions-1)
            next_state, reward, done,_, _ = env.step(action)
            episode_rewards.append(reward)

            if done:
                break
                
            state = next_state

        episode_rewards_history.append(sum(episode_rewards))
    return episode_rewards_history

env = gym.make('LunarLander-v2',render_mode = "rgb_array")
episode_rewards_history_rand = random_lunar(env, num_episodes=2000, max_steps_per_episode=1000)

os.system('say "Finished"')


/Users/luisweekes/anaconda3/envs/myenv/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


NameError: name 'policy_net' is not defined

In [ ]:
#REINFORCE 

import gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
import timeit

#from gym.wrappers.record_video import RecordVideo
start_time = timeit.default_timer()

# Define the policy network
class PolicyNetwork(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=256):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, action_size)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        action_probs = torch.softmax(self.fc2(x), dim=-1)
        return action_probs

# Reinforce algorithm
def reinforce(env, num_episodes, max_steps_per_episode, gamma=0.99, lr=0.001):
    policy_net = PolicyNetwork(state_size=env.observation_space.shape[0],
                               action_size=env.action_space.n)
    optimizer = optim.Adam(policy_net.parameters(), lr=lr)

    episode_rewards_history = []
    #last_episode_video_path = './Documents/Reinforcement_Learning/RL_proj/rl-group-assignment'
    
    for episode in range(num_episodes):
        state = env.reset()[0]
        episode_rewards = []
        log_probs = []

        #if episode == num_episodes-1:
         #   env.start_video_recorder()
        #    pass
        
        
        for step in range(max_steps_per_episode):


            state_tensor = torch.from_numpy(state).float().unsqueeze(0)
            action_probs = policy_net(state_tensor)
            action_dist = torch.distributions.Categorical(action_probs)
            action = action_dist.sample()
            log_prob = action_dist.log_prob(action)
            
            #print(env.step(action.item()))
            next_state, reward, done,_, _ = env.step(action.item())
            episode_rewards.append(reward)
            log_probs.append(log_prob)
            
            if done:
                break
                
            state = next_state

        #if episode == num_episodes-1:
        #    pass
         ##   env.close_video_recorder()

        # Compute the discounted cumulative reward
        discounted_rewards = []
        R = 0
        for r in episode_rewards[::-1]:
            R = r + gamma * R
            discounted_rewards.insert(0, R)

        episode_rewards_history.append(sum(episode_rewards))
        
        # Normalize the discounted rewards
        discounted_rewards = torch.tensor(discounted_rewards)
        discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-9)
        
        # Compute the policy gradient and update the policy network
        policy_losses = []
        for log_prob, reward in zip(log_probs, discounted_rewards):
            policy_losses.append(-log_prob * reward)
        
        policy_loss = torch.stack(policy_losses).sum()
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
    #env.close()   
    return policy_net, episode_rewards_history

# Train the policy network
env = gym.make('LunarLander-v2',render_mode = "rgb_array")
#env = RecordVideo(env = env, video_folder="./videos",name_prefix='Last_episode_video')

policy_net, episode_rewards_history_reinforce = reinforce(env, num_episodes=2000, max_steps_per_episode=1000)

end_time = timeit.default_timer()
execution_time = end_time - start_time
print("Execution time:", execution_time)

def running_mean(x):
    N=50
    kernel = np.ones(N)
    conv_len = x.shape[0]-N
    y = np.zeros(conv_len)
    for i in range(conv_len):
        y[i] = kernel @ x[i:i+N]
        y[i] /= N
    return y

episode_rewards_history_reinforce = np.array(episode_rewards_history_reinforce)
reinforce_mean_rewards = running_mean(episode_rewards_history_reinforce)

os.system('say "Finished"')


In [ ]:
#DQN

import torch
import torch.nn as nn
import torch.optim as optim

import random
import numpy as np
import gym
import gym.spaces as sp
from tqdm import trange
from time import sleep
from collections import namedtuple, deque
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#%% Policy network
class QNet(nn.Module):
    # Policy Network
    def __init__(self, n_states, n_actions, n_hidden=64):
        super(QNet, self).__init__()

        self.fc = nn.Sequential(
            nn.Linear(n_states, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_actions)
            )

    def forward(self, x):
        return self.fc(x)

#%% dqn    
class DQN():
    def __init__(self, n_states, n_actions, batch_size=64, lr=1e-4, gamma=0.99, mem_size=int(1e5), learn_step=5, tau=1e-3):
        self.n_states = n_states
        self.n_actions = n_actions
        self.batch_size = batch_size
        self.gamma = gamma
        self.learn_step = learn_step
        self.tau = tau

        # model
        self.net_eval = QNet(n_states, n_actions).to(device)
        self.net_target = QNet(n_states, n_actions).to(device)
        self.optimizer = optim.Adam(self.net_eval.parameters(), lr=lr)
        self.criterion = nn.MSELoss()

        # memory
        self.memory = ReplayBuffer(n_actions, mem_size, batch_size)
        self.counter = 0    # update cycle counter

    def getAction(self, state, epsilon):
        #print('state1: ', state)
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        #print('state2: ', state)

        self.net_eval.eval()
        with torch.no_grad():
            action_values = self.net_eval(state)
        self.net_eval.train()

        # epsilon-greedy
        if random.random() < epsilon:
            action = random.choice(np.arange(self.n_actions))
        else:
            action = np.argmax(action_values.cpu().data.numpy())

        return action

    def save2memory(self, state, action, reward, next_state, done):
        self.memory.add(state, action, reward, next_state, done)

        self.counter += 1
        if self.counter % self.learn_step == 0:
            if len(self.memory) >= self.batch_size:
                experiences = self.memory.sample()
                self.learn(experiences)

    def learn(self, experiences):
        states, actions, rewards, next_states, dones = experiences

        q_target = self.net_target(next_states).detach().max(axis=1)[0].unsqueeze(1)
        y_j = rewards + self.gamma * q_target * (1 - dones)          # target, if terminal then y_j = rewards
        q_eval = self.net_eval(states).gather(1, actions)

        # loss backprop
        loss = self.criterion(q_eval, y_j)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # soft update target network
        self.softUpdate()

    def softUpdate(self):
        for eval_param, target_param in zip(self.net_eval.parameters(), self.net_target.parameters()):
            target_param.data.copy_(self.tau*eval_param.data + (1.0-self.tau)*target_param.data)


class ReplayBuffer():
    def __init__(self, n_actions, memory_size, batch_size):
        self.n_actions = n_actions
        self.batch_size = batch_size
        self.memory = deque(maxlen = memory_size)
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])

    def __len__(self):
        return len(self.memory)

    def add(self, state, action, reward, next_state, done):
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)

    def sample(self):
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)

def running_mean(iterations, score_hist):
    window_size = 100 #for the running mean

    if iterations == 0:
        score_avg = score_hist[0]
    elif iterations < window_size:
        score_avg = np.mean(score_hist[:iterations])
    else:
        score_avg = np.mean(score_hist[iterations - window_size : iterations])


    return score_avg

def train(env, agent, n_episodes=2000, max_steps=1000, eps_start=1.0, eps_end=0.1, eps_decay=0.995, target=200, chkpt=False):
    score_hist = []
    mean_rewards = []
    epsilon = eps_start

    bar_format = '{l_bar}{bar:10}| {n:4}/{total_fmt} [{elapsed:>7}<{remaining:>7}, {rate_fmt}{postfix}]'
    # bar_format = '{l_bar}{bar:10}{r_bar}'
    pbar = trange(n_episodes, unit="ep", bar_format=bar_format, ascii=True)
    for idx_epi in pbar:
        state = env.reset()[0]
        score = 0
        for idx_step in range(max_steps):
            action = agent.getAction(state, epsilon)
            movement = env.step(action)
            next_state, reward, done, _ = movement[0], movement[1], movement[2], movement[4]
            #reward = np.clip(reward, -1, 1)
            agent.save2memory(state, action, reward, next_state, done)
            state = next_state
            score += reward

            if done:
                break

        score_hist.append(score)
        epsilon = max(eps_end, epsilon*eps_decay)

        score_avg = running_mean(idx_epi, score_hist)
        mean_rewards.append(score_avg)

        pbar.set_postfix_str(f"Score: {score: 7.2f}, 100 score avg: {score_avg: 7.2f}")
        pbar.update(0)

        # if (idx_epi+1) % 100 == 0:
        #     print(" ")
        #     sleep(0.1)

        # Early stop
        if len(score_hist) >= 100:
            if score_avg >= target:
                break

    if (idx_epi+1) < n_episodes:
        print("\nTarget Reached!")
    else:
        print("\nDone!")
        
    if chkpt:
        torch.save(agent.net_eval.state_dict(), 'checkpoint.pth')

    return score_hist, mean_rewards

#%% Test Lunar Lander
def testLander(env, agent, loop=3):
    for i in range(loop):
        state = env.reset()
        for idx_step in range(500):
            action = agent.getAction(state, epsilon=0)
            env.render()
            state, reward, done, _ = env.step(action)
            if done:
                break
    env.close()
"""
def plotScore(scores, mean_rewards):
    plt.figure(figsize=(15,7))
    plt.ylabel("Rewards",fontsize=12)
    plt.xlabel("Episodes",fontsize=12)
    plt.plot(scores, 
             color='gray', 
             linewidth=1)
    plt.scatter(np.arange(len(scores)),
                scores,
                color='green',
                linewidth=0.3,
                label = 'Episode Rewards')
    plt.plot(mean_rewards,
             color='blue',
             linewidth=3,
             label = 'Running Average Score of DQN Policy')

    plt.title('Rewards by Episode For DQN in Lunar Lander')
    plt.legend()
    plt.show()

    """plt.figure()
    plt.plot(scores)
    plt.title("Score History")
    plt.xlabel("Episodes")
    plt.show()"""
"""

BATCH_SIZE = 128
LR = 1e-3
EPISODES = 1500
TARGET_SCORE = 250.     # early training stop at avg score of last 100 episodes
GAMMA = 0.99            # discount factor
MEMORY_SIZE = 10000     # max memory buffer size
LEARN_STEP = 5          # how often to learn
TAU = 1e-3              # for soft update of target parameters
SAVE_CHKPT = False      # save trained network .pth file


#env = gym.make('LunarLander-v2')
env = gym.make("LunarLander-v2", render_mode="human")
num_states = env.observation_space.shape[0]
num_actions = env.action_space.n
agent = DQN(
    n_states = num_states,
    n_actions = num_actions,
    batch_size = BATCH_SIZE,
    lr = LR,
    gamma = GAMMA,
    mem_size = MEMORY_SIZE,
    learn_step = LEARN_STEP,
    tau = TAU,
    )
score_hist, mean_rewards = train(env, agent, n_episodes=EPISODES, target=TARGET_SCORE, chkpt=SAVE_CHKPT)
#plotScore(score_hist, mean_rewards)

if str(device) == "cuda":
    torch.cuda.empty_cache()
